# VacationPy


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
import sys
#reference folder key file is in
sys.path.append("..")



# Import API key
from api_keys import g_key

### Load and Store Weatherpy Results into DataFrame


In [5]:
#upload weather data and load into dataframe
file = "../WeatherPy/cityweatherdata.csv"
weather_df = pd.read_csv(file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,aanekoski,62.6946,25.8180,43.00,81,90,4.61,FI,1619185732
1,abaza,52.6550,90.0928,30.85,62,62,1.21,RU,1619185732
2,adrar,25.7500,-1.0000,102.83,4,0,11.07,DZ,1619185733
3,adre,13.4667,22.2000,100.51,8,100,6.40,TD,1619185733
4,agadez,19.7500,10.2500,103.57,6,100,9.26,NE,1619185733


### Humidity Heatmap


In [6]:
#configure gmaps key
gmaps.configure(api_key=g_key)

#set and display heatmap of humidity
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    weather_df[['Lat','Lng']], weights = weather_df['Humidity'],
    max_intensity=weather_df['Humidity'].max()
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria


In [7]:
#sorting ideal weather dataframe

ideal_weather = weather_df.query('`Max Temp`<80 & `Max Temp`>70 & `Wind Speed`<10 &Cloudiness==0')
ideal_weather


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,dinar,38.0650,30.1656,75.20,22,0,5.75,TR,1619185523
71,nurobod,40.9056,69.8023,71.60,26,0,9.22,UZ,1619185751
146,champerico,14.3000,-91.9167,75.20,88,0,3.42,GT,1619185768
239,santa cruz,10.2605,-85.5851,73.40,83,0,4.61,CR,1619185791
314,katobu,-4.9408,122.5276,75.00,92,0,1.90,ID,1619185810
349,noumea,-22.2763,166.4572,78.01,88,0,2.30,NC,1619185554
355,port hedland,-20.3167,118.5667,73.40,53,0,6.91,AU,1619185819
399,aparecida do taboado,-20.0867,-51.0936,79.54,41,0,5.48,BR,1619185833
431,cosmopolis,-22.6458,-47.1961,75.99,48,0,4.63,BR,1619185844
440,guaruja,-23.9931,-46.2564,75.20,64,0,4.61,BR,1619185847


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#empty hotel_df
hotel_df = pd.DataFrame(columns = ["Hotel Name"])

In [9]:
#set base url plus params dictionary
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius":5000,
    "keyword": "hotel",
    "type":"lodging",
    "key": g_key,
}

In [10]:
#loop through ideal weather cities
for index, row in ideal_weather.iterrows():


# add location to params dict
    params['location'] = f"{str(row['Lat'])},{str(row['Lng'])}"


# make url and make API request
    response = requests.get(base_url, params=params).json()
    
# extract results
    results = response['results']

#fill in hotel_df with hotel information
    try:
       
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
        hotel_df.loc[index, 'City'] = row['City']
        hotel_df.loc[index, 'Country'] = row['Country']       
        
    except:
        print(f"No Hotels near {row['City']}")


No Hotels near nurobod
No Hotels near katobu
No Hotels near cosmopolis


In [11]:
#display hotels
hotel_df

,Hotel Name,Lat,Lng,City,Country
24,DİNAR AKDEM OTEL,38.066129,30.166167,dinar,TR
146,"Hotel y restaurante ""El Diamante""",14.292408,-91.913310,champerico,GT
239,Hotel Diria,10.267159,-85.587623,santa cruz,CR
349,Hilton Noumea La Promenade Residences,-22.301809,166.446066,noumea,NC
355,Hedland Hotel,-20.308700,118.610356,port hedland,AU
399,Emmis Park Hotel,-20.072897,-51.100456,aparecida do taboado,BR
440,Hotel Doral Guarujá,-23.985241,-46.221868,guaruja,BR
446,Hotel Promenade Fusion Itaguaí,-22.871299,-43.768432,itaguai,BR
495,Pousada Porto Prime,-21.638252,-41.020602,sao joao da barra,BR


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))